In [38]:
import pandas as pd
import seaborn as sns
from sklearn import metrics

In [2]:
#reading each xlsx and changing columns to match to then concat
df2014 = pd.read_excel('data/ttc-streetcar-delay-data-2014.xlsx')
df2015 = pd.read_excel('data/ttc-streetcar-delay-data-2015.xlsx')
df2016 = pd.read_excel('data/ttc-streetcar-delay-data-2016.xlsx')
df2017 = pd.read_excel('data/ttc-streetcar-delay-data-2017.xlsx')
df2018 = pd.read_excel('data/ttc-streetcar-delay-data-2018.xlsx')
df2019 = pd.read_excel('data/ttc-streetcar-delay-data-2019.xlsx')
df2020 = pd.read_excel('data/ttc-streetcar-delay-data-2020_final.xlsx').rename(columns={'Delay':'Min Delay',
                                                                                  'Gap':'Min Gap'})
df2021 = pd.read_excel('data/ttc-streetcar-delay-data-jan-2021-to-august-2021.xlsx').rename(columns={'Bound':'Direction',
                                                                                               'Date':'Report Date',
                                                                                               'Line':'Route'})

df2014['Time'] = pd.to_datetime(df2014['Time'], format='%H:%M:%S').dt.time
df2015['Time'] = pd.to_datetime(df2015['Time'], format='%H:%M:%S').dt.time
df2016['Time'] = pd.to_datetime(df2016['Time'], format='%H:%M:%S').dt.time
df2017['Time'] = pd.to_datetime(df2017['Time'], format='%H:%M:%S').dt.time
df2018['Time'] = pd.to_datetime(df2018['Time'], format='%H:%M:%S').dt.time
df2019['Time'] = pd.to_datetime(df2019['Time'], format='%H:%M:%S').dt.time
df2020['Time'] = pd.to_datetime(df2020['Time'], format='%H:%M:%S').dt.time
df2021['Time'] = pd.to_datetime(df2021['Time'], format='%H:%M:%S').dt.time

frames = [df2014, df2015, df2016, df2017, df2018, df2019, df2020, df2021]

In [5]:
df = pd.concat(frames)
df = df.rename(columns={'Report Date':'Report_Date','Min Delay':'Min_Delay','Min Gap':'Min_Gap',
                       'Round Time':'Round_Time'})

## Removing NAs

In [6]:
#summing na values
df.isna().sum()

Report_Date      0
Route            2
Time             0
Day              0
Location        21
Incident         0
Min_Delay        3
Min_Gap          5
Direction       98
Vehicle        503
Round_Time       0
dtype: int64

In [7]:
#removing vehicle as a column
df = df.drop('Vehicle', axis=1)

In [8]:
#removing na values
df = df.dropna()#.sample(frac = 1)
df = df.drop('Min_Gap', axis=1)

In [9]:
df = df.loc[~((df['Min_Delay'] == 0))]
df["Location"] = df['Location'].str.replace('[^\w\s]','')
def clean_text(text):
    text = text.lower()
    text = text.replace(" ", "_")
    return text

df['Location'] = df['Location'].astype(str).apply(clean_text)
df['Incident'] = df['Incident'].astype(str).apply(clean_text)

/var/folders/fm/xwcj8f4d2xl395kr3dnsf8z40000gn/T/ipykernel_98991/39181628.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  df["Location"] = df['Location'].str.replace('[^\w\s]','')


In [10]:
df

,Report_Date,Route,Time,Day,Location,Incident,Min_Delay,Direction,Round_Time
0,2014-01-02,505,06:31:00,Thursday,dundas_and_roncesvalles,late_leaving_garage,4.0,E/B,06:30:00
1,2014-01-02,504,12:43:00,Thursday,king_and_shaw,utilized_off_route,20.0,E/B,12:30:00
2,2014-01-02,501,14:01:00,Thursday,kingston_road_and_bingham,held_by,13.0,W/B,14:00:00
3,2014-01-02,504,14:22:00,Thursday,king_st_and_roncesvalles_ave,investigation,7.0,W/B,14:30:00
4,2014-01-02,504,16:42:00,Thursday,king_and_bathurst,utilized_off_route,3.0,E/B,16:30:00
...,...,...,...,...,...,...,...,...,...
1012,2021-01-31,512,22:10:00,Sunday,gunns_loop,mechanical,5.0,E,22:00:00
1013,2021-01-31,501,22:43:00,Sunday,queen_and_broadview,emergency_services,11.0,W,22:30:00
1014,2021-01-31,504,00:15:00,Sunday,king_and_spadina,mechanical,3.0,W,00:30:00
1016,2021-01-31,501,00:46:00,Sunday,queen_and_church,held_by,13.0,W,01:00:00


## Date Continuous

In [11]:
from datetime import timedelta

time_in_minutes = []
time_in_hours = []

for time in df['Time']:
    delta = timedelta(hours=int(time.strftime('%H')), 
                      minutes=int(time.strftime('%M')), 
                      seconds=int(time.strftime('%S')))
    total_seconds = delta.total_seconds()
    time_in_minutes.append(total_seconds/60)
    time_in_hours.append(total_seconds/60/60)

df['Time_in_hours'] = time_in_hours

In [61]:
time_category = []

for i in df['Time_in_hours']:
    if 5 <= i < 12:
        i = 'morning'
        time_category.append(i)

for i in df['Time_in_hours']:
    if 12 <= i < 17:
        i = 'afternoon'
        time_category.append(i)
        
for i in df['Time_in_hours']:
    if 17 <= i < 21:
        i = 'evening'
        time_category.append(i)

for i in df['Time_in_hours']:
    if 21 <= i <= 24:
        i = 'night'
        time_category.append(i)

for i in df['Time_in_hours']:
    if 0 <= i < 5:
        i = 'night'
        time_category.append(i)

df['time_category'] = time_category
df

,Report_Date,Route,Time,Day,Location,Incident,Min_Delay,Direction,Round_Time,Time_in_hours,time_category
0,2014-01-02,505,06:31:00,Thursday,dundas_and_roncesvalles,late_leaving_garage,4.0,E/B,06:30:00,6.516667,morning
1,2014-01-02,504,12:43:00,Thursday,king_and_shaw,utilized_off_route,20.0,E/B,12:30:00,12.716667,morning
2,2014-01-02,501,14:01:00,Thursday,kingston_road_and_bingham,held_by,13.0,W/B,14:00:00,14.016667,morning
3,2014-01-02,504,14:22:00,Thursday,king_st_and_roncesvalles_ave,investigation,7.0,W/B,14:30:00,14.366667,morning
4,2014-01-02,504,16:42:00,Thursday,king_and_bathurst,utilized_off_route,3.0,E/B,16:30:00,16.700000,morning
...,...,...,...,...,...,...,...,...,...,...,...
1012,2021-01-31,512,22:10:00,Sunday,gunns_loop,mechanical,5.0,E,22:00:00,22.166667,night
1013,2021-01-31,501,22:43:00,Sunday,queen_and_broadview,emergency_services,11.0,W,22:30:00,22.716667,night
1014,2021-01-31,504,00:15:00,Sunday,king_and_spadina,mechanical,3.0,W,00:30:00,0.250000,night
1016,2021-01-31,501,00:46:00,Sunday,queen_and_church,held_by,13.0,W,01:00:00,0.766667,night


## Naive Bayes Classifier

In [82]:
encoding = df.drop(['Report_Date', 'Min_Delay',
                   'Round_Time', 'Time_in_hours', 'Time', 'time_category'], axis=1)
encoding

,Route,Day,Location,Incident,Direction
0,505,Thursday,dundas_and_roncesvalles,late_leaving_garage,E/B
1,504,Thursday,king_and_shaw,utilized_off_route,E/B
2,501,Thursday,kingston_road_and_bingham,held_by,W/B
3,504,Thursday,king_st_and_roncesvalles_ave,investigation,W/B
4,504,Thursday,king_and_bathurst,utilized_off_route,E/B
...,...,...,...,...,...
1012,512,Sunday,gunns_loop,mechanical,E
1013,501,Sunday,queen_and_broadview,emergency_services,W
1014,504,Sunday,king_and_spadina,mechanical,W
1016,501,Sunday,queen_and_church,held_by,W


In [83]:
#encoded df
dummy_data = df

for i in encoding:
    dummies = pd.get_dummies(encoding[i])
    dummy_data = pd.concat([dummy_data, dummies], axis=1)
    dummy_data = dummy_data.drop(i, axis=1)

dummy_data = dummy_data.drop(['Report_Date', 'Round_Time', 'Time', 'Time_in_hours'], axis=1)

dummy_data['Min_Delay'] = (dummy_data['Min_Delay'] - dummy_data['Min_Delay'].min()) / (dummy_data['Min_Delay'].max() - dummy_data['Min_Delay'].min())

dummy_data

,Min_Delay,time_category,1,2,3,5,7,8,10,11,...,WB,btw,bw,eb,ee,ew,n/b,r,s,wb
0,0.005008,morning,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0.031720,morning,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0.020033,morning,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0.010017,morning,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0.003339,morning,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1012,0.006678,night,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1013,0.016694,night,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1014,0.003339,night,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1016,0.020033,night,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [84]:
# creating the holdout dataset

holdout = dummy_data.sample(frac = 0.1, random_state = 1)
dummy_data = dummy_data.drop(holdout.index)

In [91]:
# baseline

(testing['time_category']==training['time_category'].mode().tolist()[0]).mean()

0.43921139101861995

In [85]:
# split df into training and testing

from sklearn.model_selection import train_test_split

t = 'time_category'
dummy_data[t]
training, testing = train_test_split(dummy_data, test_size = .2, stratify = dummy_data[t], random_state = 1)

In [87]:
# build model

from sklearn.metrics import f1_score, accuracy_score
from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB()
classifier.fit(training.drop('time_category', axis = 1), training['time_category'])
pred = classifier.predict(testing.drop('time_category', axis = 1))

/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/validation.py:1673: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/validation.py:1673: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(


In [88]:
# compute the accuracy score of df classifier

training_score = metrics.accuracy_score(testing['time_category'], pred)
print("accuracy:   %0.3f" % training_score)

accuracy:   0.554


In [89]:
# apply model to holdout dataset

pred_holdout = classifier.predict(holdout.drop('time_category', axis = 1))

# compute the performance measures of holdout

estimated_accuracy = metrics.accuracy_score(holdout['time_category'], pred_holdout)
print("accuracy:   %0.3f" % estimated_accuracy)

#pd.Series(estimated_accuracy).to_csv('ea.csv', index=False, header=False)

accuracy:   0.603


/opt/homebrew/lib/python3.9/site-packages/sklearn/utils/validation.py:1673: FutureWarning: Feature names only support names that are all strings. Got feature names with dtypes: ['int', 'str']. An error will be raised in 1.2.
  warnings.warn(
